#### Global & Local Modules

In [1]:
import os
from itertools import permutations 
import sys
import numpy as np
import scipy.sparse as sp
from tqdm import tqdm
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2
from rgcn import GaussianGraphConvolution, kl_reg

Using TensorFlow backend.


#### Vendor Modules

In [2]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS
# import vendor modules
from kegra.layers.graph import GraphConvolution
from kegra.utils import load_data, preprocess_adj, get_splits, evaluate_preds

# Data

In [3]:
def attack_edges(attack_ratio: float) -> 'callable':
    """
    Create an edge attack function with given attack ratio.

    Args:
        attack_ratio: the ratio of noise to clean edges

    Returns:
        a callable function for attacking a graph

    """
    def _attack_edges(edges: np.ndarray) -> np.ndarray:
        f"""
        Attack the edges of the given list of edges.

        Args:
            edges: the ndarray of tuples representing edges to attack

        Returns:
            a new ndarray of edges with random edges added

        Note:
            the ratio of noise to clean edges is {attack_ratio}

        """
        # create the set of all possible edges
        all_edges = set(permutations(list(range(edges.max())), 2))
        # convert the ndarray of edges to a set of tuples
        edges = set(map(tuple, edges))
        # create the set of edges to sample by subtracting the existing
        # edges from the set of possible edges
        sample_edges = all_edges - edges
        # select random edges without replacement based on the noise ratio
        attack_edges = np.random.choice(list(range(len(sample_edges))), 
            size=int(len(edges) * attack_ratio), 
            replace=False)
        attack_edges = [edge for (i, edge) in enumerate(list(sample_edges)) if i in attack_edges]
        # create the new ndarray of edges with the attack edges added
        edges = list(edges) + attack_edges
        return np.array(edges)
    return _attack_edges

In [4]:
X, A, y = load_data('data/cora/', dataset='cora', attack_edges=attack_edges(0))
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask, val_mask, test_mask = get_splits(y)

In [5]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [7]:
y

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]], dtype=int32)

# Model

## Input Layers

In [8]:
X_in = Input(shape=(X.shape[1], ), name='features')
X_in

<tf.Tensor 'features:0' shape=(?, 1433) dtype=float32>

In [9]:
A_in = Input(shape=(None, None), batch_shape=(None, None), sparse=True, name='graph')
A_in

## Training Loop

In [10]:
def train(model, epochs=200, patience=10):
    """
    Train the given model.

    Args:
        model: the model to train
        epochs: the maximum number of training epochs
        patience: the number of patience epochs for early stoping

    Returns:
        a tuple of the testing loss and accuracy

    """
    history = model.fit([X, A], y_train,
        sample_weight=train_mask,
        batch_size=A.shape[0],
        epochs=epochs,
        shuffle=False,
        callbacks=[
            EarlyStopping(monitor='val_loss', patience=patience),
        ],
        validation_data = ([X, A], y_val, val_mask)
    )
    val = model.evaluate([X, A], y_test, 
        sample_weight=test_mask, 
        batch_size=A.shape[0],
    )
    return val
#     wait = 0
#     preds = None
#     best_val_loss = 99999
#     # Fit
#     progress = tqdm(range(1, epochs + 1))
#     for epoch in progress:
#         # Single training iteration (we mask nodes without labels for loss calculation)
#         model.fit([X, A], y_train, 
#             sample_weight=train_mask,
#             batch_size=A.shape[0], 
#             epochs=1, 
#             shuffle=False, 
#             verbose=0)
#         # Predict on full dataset
#         preds = model.predict([X, A], batch_size=A.shape[0])
#         # Train / validation scores
#         train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val], [idx_train, idx_val])
#         # update the progress bar
#         progress.set_postfix(train_acc=train_val_acc[0], val_acc=train_val_acc[1])
#         # Early stopping
#         if train_val_loss[1] < best_val_loss:  # better loss than best
#             best_val_loss = train_val_loss[1]
#             wait = 0
#         elif wait >= patience:  # early stopping
#             break
#         else:  # waiting for better loss
#             wait += 1
#     progress.close()
#     # return test set evaluation metrics
#     return evaluate_preds(preds, [y_test], [idx_test])

## GCN (baseline)

In [11]:
# create the layers
H = Dropout(0.5)(X_in)
H = GraphConvolution(32, 1, 
    activation='relu', 
    kernel_regularizer=l2(5e-4)
)([H, A_in])
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], 1, 
    activation='softmax'
)([H, A_in])
# create the model
gcn = Model(inputs=[X_in, A_in], outputs=Y)
gcn.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), weighted_metrics=['acc'])
gcn.summary()



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
features (InputLayer)           (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1433)         0           features[0][0]                   
__________________________________________________________________________________________________
graph (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
graph_convolution_1 (GraphConvo (None, 32)           45888       dropout_1[0][0]         

In [12]:
train(gcn)[1]

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2708 samples, validate on 2708 samples
Epoch 1/200
2708/2708 [==============================] - 0s 86us/step - loss: 1.9772 - weighted_acc: 0.1857 - val_loss: 1.9588 - val_weighted_acc: 0.3500
Epoch 2/200
2708/2708 [==============================] - 0s 19us/step - loss: 1.9583 - weighted_acc: 0.3071 - val_loss: 1.9389 - val_weighted_acc: 0.3500
Epoch 3/200
2708/2708 [==============================] - 0s 20us/step - loss: 1.9386 - weighted_acc: 0.3000 - val_loss: 1.9175 - val_weighted_acc: 0.3500
Epoch 4/200
2708/2708 [==============================] - 0s 19us/step - loss: 1.9182 - weighted_acc: 0.2929 - val_loss: 1.8985 - val_weighted_acc: 0.3500
Epoch 5/200
2708/2708 [==============================] - 0s 18us/step - loss: 1.8991 - weighted_acc: 0.2929 - val_loss: 1.8807 - val_weighted_acc: 0.3500
Epoch 6/200
2708/2708 [==============================] - 0s 17us/step - loss: 1.8808 - w

Epoch 51/200
2708/2708 [==============================] - 0s 17us/step - loss: 1.3123 - weighted_acc: 0.7214 - val_loss: 1.5002 - val_weighted_acc: 0.6800
Epoch 52/200
2708/2708 [==============================] - 0s 17us/step - loss: 1.3250 - weighted_acc: 0.7643 - val_loss: 1.4911 - val_weighted_acc: 0.6933
Epoch 53/200
2708/2708 [==============================] - 0s 17us/step - loss: 1.3077 - weighted_acc: 0.7714 - val_loss: 1.4825 - val_weighted_acc: 0.6967
Epoch 54/200
2708/2708 [==============================] - 0s 15us/step - loss: 1.3103 - weighted_acc: 0.7643 - val_loss: 1.4743 - val_weighted_acc: 0.6867
Epoch 55/200
2708/2708 [==============================] - 0s 15us/step - loss: 1.2669 - weighted_acc: 0.7929 - val_loss: 1.4664 - val_weighted_acc: 0.7033
Epoch 56/200
2708/2708 [==============================] - 0s 17us/step - loss: 1.2967 - weighted_acc: 0.7214 - val_loss: 1.4593 - val_weighted_acc: 0.7067
Epoch 57/200
2708/2708 [==============================] - 0s 16us/step

Epoch 104/200
2708/2708 [==============================] - 0s 16us/step - loss: 0.9529 - weighted_acc: 0.8857 - val_loss: 1.1843 - val_weighted_acc: 0.7933
Epoch 105/200
2708/2708 [==============================] - 0s 17us/step - loss: 0.9493 - weighted_acc: 0.9000 - val_loss: 1.1793 - val_weighted_acc: 0.8067
Epoch 106/200
2708/2708 [==============================] - 0s 21us/step - loss: 0.9379 - weighted_acc: 0.8929 - val_loss: 1.1750 - val_weighted_acc: 0.8033
Epoch 107/200
2708/2708 [==============================] - 0s 20us/step - loss: 0.8800 - weighted_acc: 0.9071 - val_loss: 1.1713 - val_weighted_acc: 0.8067
Epoch 108/200
2708/2708 [==============================] - 0s 20us/step - loss: 0.9381 - weighted_acc: 0.8786 - val_loss: 1.1673 - val_weighted_acc: 0.8133
Epoch 109/200
2708/2708 [==============================] - 0s 19us/step - loss: 0.9649 - weighted_acc: 0.8643 - val_loss: 1.1656 - val_weighted_acc: 0.8133
Epoch 110/200
2708/2708 [==============================] - 0s 20

Epoch 157/200
2708/2708 [==============================] - 0s 18us/step - loss: 0.8009 - weighted_acc: 0.9071 - val_loss: 1.0521 - val_weighted_acc: 0.8300
Epoch 158/200
2708/2708 [==============================] - 0s 19us/step - loss: 0.7748 - weighted_acc: 0.9357 - val_loss: 1.0496 - val_weighted_acc: 0.8333
Epoch 159/200
2708/2708 [==============================] - 0s 17us/step - loss: 0.7267 - weighted_acc: 0.9214 - val_loss: 1.0469 - val_weighted_acc: 0.8333
Epoch 160/200
2708/2708 [==============================] - 0s 18us/step - loss: 0.7410 - weighted_acc: 0.9214 - val_loss: 1.0449 - val_weighted_acc: 0.8367
Epoch 161/200
2708/2708 [==============================] - 0s 18us/step - loss: 0.7639 - weighted_acc: 0.9500 - val_loss: 1.0420 - val_weighted_acc: 0.8333
Epoch 162/200
2708/2708 [==============================] - 0s 18us/step - loss: 0.8080 - weighted_acc: 0.9357 - val_loss: 1.0418 - val_weighted_acc: 0.8300
Epoch 163/200
2708/2708 [==============================] - 0s 18

0.8149991035461426

## Gaussian Graph Convolution

In [13]:
# create the layers
H = Dropout(0.6)(X_in)
M1, S1 = GaussianGraphConvolution(16,
    is_first=True,
    dropout=0.6
)([H, A_in])
Y = GaussianGraphConvolution(y.shape[1],
    is_last=True,
    last_activation='softmax',
)([M1, S1, A_in])
# create the model
rgcn = Model(inputs=[X_in, A_in], outputs=Y)
rgcn.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), weighted_metrics=['acc'])
rgcn.add_loss(kl_reg(M1, S1, 5e-4), [M1, S1])
rgcn.summary()

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
features (InputLayer)           (None, 1433)         0                                        

In [14]:
train(rgcn)[1]

Train on 2708 samples, validate on 2708 samples
Epoch 1/200
2708/2708 [==============================] - 0s 112us/step - loss: 1.9845 - weighted_acc: 0.2214 - val_loss: nan - val_weighted_acc: 0.1267
Epoch 2/200
2708/2708 [==============================] - 0s 18us/step - loss: nan - weighted_acc: 0.1500 - val_loss: nan - val_weighted_acc: 0.1267
Epoch 3/200
2708/2708 [==============================] - 0s 20us/step - loss: nan - weighted_acc: 0.1500 - val_loss: nan - val_weighted_acc: 0.1267
Epoch 4/200
2708/2708 [==============================] - 0s 20us/step - loss: nan - weighted_acc: 0.1500 - val_loss: nan - val_weighted_acc: 0.1267
Epoch 5/200
2708/2708 [==============================] - 0s 20us/step - loss: nan - weighted_acc: 0.1500 - val_loss: nan - val_weighted_acc: 0.1267
Epoch 6/200
2708/2708 [==============================] - 0s 18us/step - loss: nan - weighted_acc: 0.1500 - val_loss: nan - val_weighted_acc: 0.1267
Epoch 7/200
2708/2708 [==============================] - 0s 

0.07199999690055847